# Course Pattern Extraction and Application

This notebook implements a two-phase approach for:
1. Extracting graph-based patterns from a course about mathematical induction
2. Using those patterns to construct knowledge graphs for specific proof examples

---

## Phase 1: Course Pattern Extraction

First, we'll extract the graph-based pattern from the mathematical induction course.

In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os

# Add the project root directory to the Python path
sys.path.append(os.path.abspath(os.path.join("../../..")))

In [2]:
from IPython.display import display, Latex
import re
from src.utils.file_utils import read_proof
from src.phase1.extract_triplets import extract_triplets
from src.utils.neo4j_utils import Neo4JUtils

# Load the course content
course_latex = read_proof("../../data/courses/addition/course_2.tex")

### Extract Course Pattern

We'll use a specialized prompt to extract the pattern of mathematical induction from the course content.

In [ ]:
SYSTEM_PROMPT = """You are an expert in mathematical calculation analysis, specializing in extracting structured knowledge graph from mathematical calculation texts. Your task is to identify quantom/detailed progress/procedural steps in a mathematical calculation process and represent them as a fine-grained knowledge graph with explicit step-by-step reasoning."""

COURSE_PATTERN_PROMPT = """
Given the following mathematical course content in LaTeX format, extract a VERY DETAILED step-by-step explanatory chain that represents the calculation process. Create a knowledge graph with fine-grained steps that shows exactly how calculations proceed from start to finish.

Focus on identifying:
1. Every individual calculation step, no matter how small (e.g., "Add 2 to both sides", "Apply distributive property", etc.)
2. The precise mathematical operations performed at each step
3. The exact sequence of operations, with clear predecessor-successor relationships
4. Intermediate results at each calculation stage
5. The mathematical justification for each step (e.g., "By the associative property", "By substituting value from step 3")

IMPORTANT: If you see examples in the course content, do not extract them as separate graphs. The examples are only included to help you understand the calculation process better. Focus only on extracting the general calculation pattern/process.

The final knowledge graph MUST:
1. Have clearly marked START node(s) (the initial problem statement)
2. Have clearly marked END node(s) (the final result)
3. Include ALL intermediate calculation steps with no gaps in reasoning
4. Form a single connected component with a clear directional flow
5. Use relationship types that precisely describe the mathematical operation performed (e.g., "applies_distributive_property", "substitutes_value", "simplifies_expression")

Extract triplets in the given form of structured output that represent this detailed calculation process.

Note:
- Entities or Nodes should be specified with short mathematical expressions.
- Relations at this mathematical calculation graph is only grounding or simple reasoning connecting one expression to another.
- Steps are as much as possible explaining all the minor steps of the reasoning of the calculation.

Course Content:
```
{proof}
```
"""


# Extract the course pattern
course_pattern = extract_triplets(
    proof=course_latex,
    custom_prompt=COURSE_PATTERN_PROMPT,
    system_message=SYSTEM_PROMPT,
)
print(course_pattern)

# Store the pattern in Neo4j
neo4j = Neo4JUtils("bolt://localhost:7687", ("neo4j", "password"))
neo4j.clean_database()

[SystemMessage(content='You are an expert in mathematical calculation analysis, specializing in extracting structured knowledge graph from mathematical calculation texts. Your task is to identify quantom/detailed progress/procedural steps in a mathematical calculation process and represent them as a fine-grained knowledge graph with explicit step-by-step reasoning.', additional_kwargs={}, response_metadata={}), HumanMessage(content='\nGiven the following mathematical course content in LaTeX format, extract a VERY DETAILED step-by-step explanatory chain that represents the calculation process. Create a knowledge graph with fine-grained steps that shows exactly how calculations proceed from start to finish.\n\nFocus on identifying:\n1. Every individual calculation step, no matter how small (e.g., "Add 2 to both sides", "Apply distributive property", etc.)\n2. The precise mathematical operations performed at each step\n3. The exact sequence of operations, with clear predecessor-successor 

## Phase 2: Proof Pattern Application

Now we'll use the extracted pattern to construct knowledge graphs for specific proof examples.

In [4]:
# Load a proof example
problem_latex = read_proof("../../data/courses/addition/example3.tex")

In [6]:
PROOF_PATTERN_APPLICATION_PROMPT = """
Given the following mathematical calculation and the given pattern of mathematical calculation extracted from the course, construct a knowledge graph that follows the given pattern.

The pattern components are:
{course_pattern}

For the given proof, extract triplets in the form <Source Entity, Relationship, Target Entity> that:
1. Follow the structure of mathematical proof pattern
2. Map to the steps identified in the course pattern
3. Capture the specific details and relationships in this proof which may be different from the course pattern

Proof:
{{proof}}
"""

# Format the prompt with course_pattern
formatted_prompt = PROOF_PATTERN_APPLICATION_PROMPT.format(
    course_pattern=course_pattern
)

# Pass the formatted prompt and proof_content to extract_triplets
proof_triplets = extract_triplets(proof=problem_latex, custom_prompt=formatted_prompt, system_message=SYSTEM_PROMPT)

# # Clean the database to remove any existing graphs
# neo4j.clean_database()

# Store the course pattern graph
neo4j.store_triplets(course_pattern, "course")

# Store the proof graph as a separate graph
neo4j.store_triplets(proof_triplets, "problem")

[SystemMessage(content='You are an expert in mathematical calculation analysis, specializing in extracting structured knowledge graph from mathematical calculation texts. Your task is to identify quantom/detailed progress/procedural steps in a mathematical calculation process and represent them as a fine-grained knowledge graph with explicit step-by-step reasoning.', additional_kwargs={}, response_metadata={}), HumanMessage(content='\nGiven the following mathematical calculation and the given pattern of mathematical calculation extracted from the course, construct a knowledge graph that follows the given pattern.\n\nThe pattern components are:\nentities=[Entity(id=\'1\', name=\'a + b\', label=\'Initial Addition\', type=\'step\', start=True, end=False), Entity(id=\'2\', name=\'a + (b-1) + 1\', label=\'First Decomposition\', type=\'step\', start=False, end=False), Entity(id=\'3\', name=\'(a + (b-2)) + 1 + 1\', label=\'Second Decomposition\', type=\'step\', start=False, end=False), Entity

In [22]:
# Load a test example
test1_latex = read_proof("../../data/courses/addition/test2.tex")

In [25]:
PROOF_PATTERN_APPLICATION_PROMPT = """
Given the following pattern of mathematical calculation extracted from the course and the given example of the calculation with the suitable answer, construct a knowledge graph that follows the given course pattern to answer to the question.

The pattern components are:
```
{course_pattern}
```

The example question and answer are:

```
{problem_latex}
```

and the extracted knowledge graph triplet to this problem is:
```
{problem_triplets}
```

Question:
```
{test1_latex}
```

For the given question, extract triplets in the form <Source Entity, Relationship, Target Entity> that:
1. Follow the structure of mathematical calculation pattern
2. Map to the steps identified in the course pattern
3. Capture the specific details and relationships in this calculation which may be different from the course pattern
4. If you see calculation of more than 2 numbers please break the problem into two problems and solve them step by step.
"""

# Format the prompt with course_pattern
formatted_prompt = PROOF_PATTERN_APPLICATION_PROMPT.format(
    course_pattern=course_pattern,
    problem_latex=problem_latex,
    problem_triplets=proof_triplets,
    test1_latex=test1_latex,
)

# Pass the formatted prompt and proof_content to extract_triplets
test_triplets = extract_triplets(
    proof=test1_latex, 
    custom_prompt=formatted_prompt, 
    system_message=SYSTEM_PROMPT
)

# Clean the database to remove any existing graphs
neo4j.clean_database()

# Store the course pattern graph
neo4j.store_triplets(course_pattern, "course")

# Store the proof graph as a separate graph
neo4j.store_triplets(proof_triplets, "problem")

# Store the test graph as a separate graph
neo4j.store_triplets(test_triplets, "test1")

[SystemMessage(content='You are an expert in mathematical calculation analysis, specializing in extracting structured knowledge graph from mathematical calculation texts. Your task is to identify quantom/detailed progress/procedural steps in a mathematical calculation process and represent them as a fine-grained knowledge graph with explicit step-by-step reasoning.', additional_kwargs={}, response_metadata={}), HumanMessage(content='\nGiven the following pattern of mathematical calculation extracted from the course and the given example of the calculation with the suitable answer, construct a knowledge graph that follows the given course pattern to answer to the question.\n\nThe pattern components are:\n```\nentities=[Entity(id=\'1\', name=\'a + b\', label=\'Initial Addition\', type=\'step\', start=True, end=False), Entity(id=\'2\', name=\'a + (b-1) + 1\', label=\'First Decomposition\', type=\'step\', start=False, end=False), Entity(id=\'3\', name=\'(a + (b-2)) + 1 + 1\', label=\'Secon

## Visualization and Analysis

You can visualize and analyze the graphs in Neo4j Browser using these queries:

### View Course Pattern:
```cypher
MATCH p=()-[r]->() WHERE r.graph_type = 'course_pattern' RETURN p
```

### View Proof Graph:
```cypher
MATCH p=()-[r]->() WHERE r.graph_type = 'proof_example' RETURN p
```

### Compare Pattern and Proof:
```cypher
MATCH p=()-[r]->() RETURN p
```